In [54]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate, FewShotPromptTemplate, ChatPromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector, MaxMarginalRelevanceExampleSelector, LengthBasedExampleSelector
from langchain_community.example_selectors import NGramOverlapExampleSelector
from langchain_openai import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

In [110]:
chat_model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [119]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

## Fixed Prompt

In [147]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
]

In [148]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human","{input}"),
    ("ai","{output}")
])

In [149]:
few_shot_prompt = FewShotChatMessagePromptTemplate(examples=examples, example_prompt=example_prompt)

print(few_shot_prompt.format())

Human: 2+2
AI: 4
Human: 2+3
AI: 5


In [150]:
chat_prompt = ChatPromptTemplate.from_messages([
                                                ("system","You're a mathematics wizard"),
                                                few_shot_prompt,
                                                ("human","{input}")
])

print(chat_prompt.format(input="10+10"))

System: You're a mathematics wizard
Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 10+10


In [124]:
chain = chat_prompt | chat_model
chain.invoke(input="10+10")

AIMessage(content='20', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 43, 'total_tokens': 44}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': 'fp_c2295e73ad', 'finish_reason': 'stop', 'logprobs': None}, id='run-d067b4f4-0291-4a00-bb92-6f7dde9c0b4c-0')

## Dynamic Prompt

In [125]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

vector_string = [" ".join(example.values()) for example in examples]
print(vector_string)

['2+2 4', '2+3 5', '2+4 6', 'What did the cow say to the moon? nothing at all', 'Write me a poem about the moon One for the moon, and one for me, who are we to talk about the moon?']


In [126]:
vector_store = Chroma.from_texts(texts=vector_string, embedding=embeddings)

In [127]:
# example_selector = SemanticSimilarityExampleSelector.from_examples(
#                                                         vectorstore_cls=vector_store,
#                                                         k=2,
#                                                         embeddings=embeddings,
#                                                         examples=examples
# )
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vector_store,
    k=2,
)

# The prompt template will load examples by passing the input do the `select_examples` method
# example_selector.select_examples({"input": "horse"})

In [132]:
example_selector.select_examples({"input":"3+10"})

[{}, {'input': '2+3', 'output': '5'}]

In [137]:
example_prompt = ChatPromptTemplate.from_messages([
                                                    ("human","{input}"),
                                                    ("ai","{output}")
                                                ])

In [138]:
similar_prompt = FewShotChatMessagePromptTemplate(
                                        example_prompt=example_prompt,
                                        example_selector=example_selector,
                                        input_variables=["input"]

)

In [139]:
similar_prompt.format(input="2+2")

KeyError: 'input'

In [142]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

examples = [
    {
        "question": "Who lived longer, Muhammad Ali or Alan Turing?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali
""",
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952
""",
    },
    {
        "question": "Who was the maternal grandfather of George Washington?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
""",
    },
    {
        "question": "Are both the directors of Jaws and Casino Royale from the same country?",
        "answer": """
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
""",
    },
]

In [143]:
example_prompt = PromptTemplate(input_variables=["question","answer"],
                                template="Question:{question}\nAnswer:{answer}")

In [146]:
example_prompt.format(**examples[0])

'Question:Who lived longer, Muhammad Ali or Alan Turing?\nAnswer:\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n'